# ViZDoom Deep RL Ablation Study

**Master's RL Course Final Project**

This notebook provides a complete template for running the ViZDoom ablation study on Google Colab.

## Contents
1. Environment Setup
2. Quick Test
3. Single Training Run
4. Ablation Study
5. Results Analysis

---
## 1. Environment Setup

Install dependencies and set up virtual display for headless rendering.

In [ ]:
# Check if running on Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print(f"Running on Colab: {IN_COLAB}")

In [ ]:
# Install system dependencies (Colab only)
if IN_COLAB:
    !apt-get update -qq
    !apt-get install -qq -y \
        libboost-all-dev libsdl2-dev libopenal-dev \
        xvfb python3-opengl
    print("System dependencies installed!")

In [ ]:
# Install Python packages
if IN_COLAB:
    !pip install -q vizdoom==1.2.4 gymnasium==1.2.3
    !pip install -q torch torchvision
    !pip install -q wandb hydra-core omegaconf
    !pip install -q matplotlib opencv-python numpy
    print("Python packages installed!")

In [ ]:
# Set up virtual display for headless rendering
import os

if IN_COLAB:
    os.system('Xvfb :1 -screen 0 1024x768x24 &')
    os.environ['DISPLAY'] = ':1'
    print("Virtual display configured!")

In [ ]:
# Clone repository (or mount Drive)
if IN_COLAB:
    # Option 1: Clone from GitHub (update with your repo URL)
    # !git clone https://github.com/username/vizdoom-ablation.git
    # %cd vizdoom-ablation
    
    # Option 2: Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive/vizdoom-ablation
    
    print(f"Working directory: {os.getcwd()}")

In [ ]:
# Add project to path
import sys
sys.path.insert(0, '.')

# Verify imports
import torch
import gymnasium
import vizdoom

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Gymnasium: {gymnasium.__version__}")
print(f"ViZDoom: {vizdoom.__version__}")

---
## 2. Quick Test

Verify that the environment and agent work correctly.

In [ ]:
# Test environment
from src.envs import make_vizdoom_env

env = make_vizdoom_env('VizdoomBasic-v0')
print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")

obs, info = env.reset()
print(f"Initial obs shape: {obs.shape}")

# Take a few random steps
for i in range(5):
    action = env.action_space.sample()
    obs, reward, term, trunc, info = env.step(action)
    print(f"Step {i+1}: action={action}, reward={reward:.2f}")

env.close()
print("\nEnvironment test passed!")

In [ ]:
# Run the full test suite
!python experiments/test_setup.py

---
## 3. Single Training Run

Train a single DQN agent on the Basic scenario.

In [ ]:
# Configure WandB (optional)
import wandb

# Login to WandB (run once)
# wandb.login()

# Or disable WandB for local runs
os.environ['WANDB_MODE'] = 'disabled'

In [ ]:
# Train DQN on Basic scenario (500 episodes)
!python experiments/train.py \
    env.scenario=VizdoomBasic-v0 \
    training.num_episodes=500 \
    logging.wandb_enabled=false \
    seed=42

In [ ]:
# View the learning curve
import matplotlib.pyplot as plt
from PIL import Image
import glob

# Find the most recent curve
curve_files = glob.glob('results/*_curve.png')
if curve_files:
    latest = max(curve_files, key=os.path.getctime)
    img = Image.open(latest)
    plt.figure(figsize=(12, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Learning Curve')
    plt.show()
else:
    print("No learning curves found yet.")

---
## 4. Ablation Study

Run the full ablation study across different configurations.

In [ ]:
# Preview ablation commands (dry run)
!python experiments/ablate.py --phase algorithms --dry-run --episodes 500

In [ ]:
# Run algorithm comparison (DQN vs Deep SARSA)
!python experiments/ablate.py \
    --phase algorithms \
    --scenarios VizdoomBasic-v0 \
    --seeds 1 2 3 \
    --episodes 500

In [ ]:
# Run learning rate ablation
!python experiments/ablate.py \
    --phase lr \
    --scenarios VizdoomBasic-v0 \
    --seeds 1 2 3 \
    --episodes 500

In [ ]:
# Run extension ablation (DDQN, Dueling, PER)
!python experiments/ablate.py \
    --phase extensions \
    --scenarios VizdoomBasic-v0 \
    --seeds 1 2 3 \
    --episodes 500

---
## 5. Results Analysis

Analyze and visualize the ablation study results.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob

# Load all CSV results
csv_files = glob.glob('results/*.csv')
print(f"Found {len(csv_files)} result files")

for f in csv_files[:5]:  # Show first 5
    print(f"  - {f}")

In [ ]:
# Load and display summary if available
summary_path = 'results/ablation_summary.csv'
if os.path.exists(summary_path):
    summary = pd.read_csv(summary_path)
    print("Ablation Summary:")
    display(summary)
else:
    print("No summary file yet. Run ablations first.")

In [ ]:
# Compare learning curves
from src.utils.plotting import plot_comparison

# Load results from different algorithms
results = {}
for f in csv_files:
    name = os.path.basename(f).replace('.csv', '')
    df = pd.read_csv(f)
    if 'reward' in df.columns:
        results[name] = df['reward'].tolist()

if results:
    # Select a few to compare
    selected = dict(list(results.items())[:5])
    plot_comparison(
        selected,
        window=50,
        title='Algorithm Comparison',
        save_path='results/comparison.png'
    )
    print("Comparison plot saved!")

In [ ]:
# Create bar chart of final rewards
from src.utils.plotting import plot_ablation_bars

# Aggregate final rewards
final_results = {}
for name, rewards in results.items():
    if len(rewards) >= 100:
        final_results[name] = {
            'mean': np.mean(rewards[-100:]),
            'std': np.std(rewards[-100:])
        }

if final_results:
    plot_ablation_bars(
        final_results,
        metric_name='Final Reward (Last 100 Episodes)',
        title='Ablation Study Results',
        save_path='results/ablation_bars.png'
    )
    print("Bar chart saved!")

---
## 6. Export for IEEE Report

Export results in formats suitable for the IEEE report.

In [ ]:
# Export WandB to CSV (if using WandB)
# Run: wandb sync
# Or download from WandB dashboard

# Create LaTeX table
if os.path.exists(summary_path):
    summary = pd.read_csv(summary_path)
    latex_table = summary.to_latex(index=False, float_format='%.2f')
    print("LaTeX Table:")
    print(latex_table)
    
    # Save to file
    with open('results/results_table.tex', 'w') as f:
        f.write(latex_table)
    print("\nSaved to results/results_table.tex")

In [ ]:
# Copy results to Google Drive for persistence
if IN_COLAB:
    !cp -r results /content/drive/MyDrive/vizdoom-ablation-results/
    print("Results copied to Google Drive!")

---
## Notes

### Training Time Estimates
- Basic scenario (500 ep): ~30 min
- TakeCover scenario (1000 ep): ~1-2 hours
- Full ablation (all phases): ~4-8 hours

### Tips
- Use GPU runtime for faster training
- Monitor Colab session limits (~12h free tier)
- Save checkpoints frequently to Drive
- Use WandB for persistent logging across sessions